In [1]:
# get unique output indices for each byte of LFSR registers

import csv
import numpy as np
import pandas as pd

csv_file_path = "./csv/output_indices.csv"
df = pd.read_csv(csv_file_path,  header=None, names=range(12), sep=',')

output_indices_128_reg = df.values

unique_byte_indices = []

# total 16 bytes in 128 registers
for byte in range(4):
    
    # get the starting position of the byte
    byte_start = byte *32
    byte_end = byte_start + 32

    # group the array by 8 rows
    byte_indices = output_indices_128_reg[byte_start:byte_end, 0:12]

    # remove the duplicated values in the array
    unique_indices = np.unique(byte_indices,  return_counts=True)

    # remove the nan values
    no_duplicates = np.delete(
        unique_indices[0], [i for i in range(unique_indices[1].size) if unique_indices[1][i] > 1 or np.isnan(unique_indices[0][i])])

    # convert to int type
    no_duplicates = no_duplicates.astype(int)

    # print("Group of output indices for byte " + str(byte) + ": ")
    # print(np.array2string(no_duplicates, separator=', '))

    # append the array into list
    unique_byte_indices.append(no_duplicates.tolist())

    
# print 2d array with values separated by ','
# for easier display in excel
for byte in unique_byte_indices:
    print(','.join(map(str, byte)))
    # each row represents the unique (non-duplicated) ouput indices of each byte
    
with open('./csv/4_byte_unique_output_indices.csv', mode='w') as csv_file:
    reg_writer = csv.writer(
        csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for row in unique_byte_indices: 
        reg_writer.writerow(row)
    
    
print("End")

114,116
34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,140,142,144,146,148,150,152,154
0,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,178,180
2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,150,152,156,158,194,196,198,200,202,204,206,208,210,212
End


In [2]:
# get number of time output differential = 1 grouped by single output index

from sage.crypto.boolean_function import BooleanFunction
from random import randint
import random
import numpy
import csv
import pandas as pd

R.<s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30, s31, s32, s33, s34, s35, s36, s37, s38, s39, s40, s41, s42, s43, s44, s45, s46, s47, s48, s49, s50, s51, s52, s53, s54, s55, s56, s57, s58, s59, s60, s61, s62, s63, s64, s65, s66, s67, s68, s69, s70, s71, s72, s73, s74, s75, s76, s77, s78, s79, s80, s81, s82, s83, s84, s85, s86, s87, s88, s89, s90, s91, s92, s93, s94, s95, s96, s97, s98, s99, s100, s101, s102, s103, s104, s105, s106, s107, s108, s109, s110, s111, s112, s113, s114, s115, s116, s117, s118, s119, s120, s121, s122, s123, s124, s125, s126, s127, b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28, b29, b30, b31, b32, b33, b34, b35, b36, b37, b38, b39, b40, b41, b42, b43, b44, b45, b46, b47, b48, b49, b50, b51, b52, b53, b54, b55, b56, b57, b58, b59, b60, b61, b62, b63, b64, b65, b66, b67, b68, b69, b70, b71, b72, b73, b74, b75, b76, b77, b78, b79, b80, b81, b82, b83, b84, b85, b86, b87, b88, b89, b90, b91, b92, b93, b94, b95, b96, b97, b98, b99, b100, b101, b102, b103, b104, b105, b106, b107, b108, b109, b110, b111, b112, b113, b114, b115, b116, b117, b118, b119, b120, b121, b122, b123, b124, b125, b126, b127, k0, k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12, k13, k14, k15, k16, k17, k18, k19, k20, k21, k22, k23, k24, k25, k26, k27, k28, k29, k30, k31, k32, k33, k34, k35, k36, k37, k38, k39, k40, k41, k42, k43, k44, k45, k46, k47, k48, k49, k50, k51, k52, k53, k54, k55, k56, k57, k58, k59, k60, k61, k62, k63, k64, k65, k66, k67, k68, k69, k70, k71, k72, k73, k74, k75, k76, k77, k78, k79, k80, k81, k82, k83, k84, k85, k86, k87, k88, k89, k90, k91, k92, k93, k94, k95, k96, k97, k98, k99, k100, k101, k102, k103, k104, k105, k106, k107, k108, k109, k110, k111, k112, k113, k114, k115, k116, k117, k118, k119, k120, k121, k122, k123, k124, k125, k126, k127> = BooleanPolynomialRing()
R.inject_variables()

C.<s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30, s31, s32, s33, s34, s35, s36, s37, s38, s39, s40, s41, s42, s43, s44, s45, s46, s47, s48, s49, s50, s51, s52, s53, s54, s55, s56, s57, s58, s59, s60, s61, s62, s63, s64, s65, s66, s67, s68, s69, s70, s71, s72, s73, s74, s75, s76, s77, s78, s79, s80, s81, s82, s83, s84, s85, s86, s87, s88, s89, s90, s91, s92, s93, s94, s95, s96, s97, s98, s99, s100, s101, s102, s103, s104, s105, s106, s107, s108, s109, s110, s111, s112, s113, s114, s115, s116, s117, s118, s119, s120, s121, s122, s123, s124, s125, s126, s127, b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28, b29, b30, b31, b32, b33, b34, b35, b36, b37, b38, b39, b40, b41, b42, b43, b44, b45, b46, b47, b48, b49, b50, b51, b52, b53, b54, b55, b56, b57, b58, b59, b60, b61, b62, b63, b64, b65, b66, b67, b68, b69, b70, b71, b72, b73, b74, b75, b76, b77, b78, b79, b80, b81, b82, b83, b84, b85, b86, b87, b88, b89, b90, b91, b92, b93, b94, b95, b96, b97, b98, b99, b100, b101, b102, b103, b104, b105, b106, b107, b108, b109, b110, b111, b112, b113, b114, b115, b116, b117, b118, b119, b120, b121, b122, b123, b124, b125, b126, b127> = BooleanPolynomialRing()
C.inject_variables()

S_256 = [R.gens()[u] for u in range(128)] #Original State at S^[256]
S = [R.gens()[u] for u in range(128)] #State at S^[256]
K = [R.gens()[u] for u in range(256,384,1)]
B = [R.gens()[u] for u in range(128,256,1)]

def grain(State_S, State_B):
    #update function of LFSR and NFSR
    feedback_L = State_S[0] + State_S[7] + State_S[38] + State_S[70] + State_S[81] + State_S[96]
    feedback_N = State_S[0] + State_B[0] + State_B[26] + State_B[56] + State_B[91] + State_B[96] + State_B[3] * State_B[67] + State_B[11] * State_B[13] + State_B[17] * State_B[18] + State_B[27] * State_B[59] + State_B[40] * State_B[48] + State_B[61] * State_B[65] + State_B[68] * State_B[84] + State_B[22] * State_B[24] * State_B[25] + State_B[70] * State_B[78] * State_B[82] + State_B[88] * State_B[92] * State_B[93] * State_B[95]

    #feedback update
    for j in range(128):
        if j<=126:
            State_S[j] = State_S[j+1]
            State_B[j] = State_B[j+1]
        else:
            State_S[127] = feedback_L
            State_B[127] = feedback_N
        
    #h function
    h = State_B[12]*State_S[8] + State_S[13]*State_S[20] + State_B[95]*State_S[42] + State_S[60]*State_S[79] + State_B[12]*State_B[95]*State_S[94]
    
    #pre-output function
    z = h + State_S[93] + State_B[2]+ State_B[15] + State_B[36] + State_B[45] + State_B[64] + State_B[73] + State_B[89]
    
    return z

# byte = 1
# val_list = [36, 38, 40, 42, 44, 46, 48, 50, 52, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]
# print ('|'.join(map(str, val_list)))

# 2D array that holds 16 groups of unique output indices of each byte
unique_output_indices = []

with open("./csv/4_byte_unique_output_indices.csv", "r", newline='') as f:
    reader = csv.reader(f, delimiter=',')
    for l in reader:
        int_list = [int(i) for i in l]
        unique_output_indices.append(int_list[:])


# total 16 bytes in 128 registers
for byte_index, row in enumerate(unique_output_indices, start=0):
# for row in unique_output_indices:
    byte = byte_index
    val_list = row  
    
    array_2d = []

    for reg in range(32):
        for output_index in range(len(val_list)): 
            faulty_state = (32 * byte) + reg
            result = [0]*(len(val_list))

            for i in range(100): 
                #Test for 100 random initial state
                #initialize the internal states with random value (random Key and IV)

                S = [R(random.randint(0,1)) for u in range(128)]
                B = [R(random.randint(0,1)) for u in range(128,256,1)]
                statef_S = [C() for u in range(128)]
                statef_B = [C() for u in range(128,256,1)]
                for l in range(128):
                    statef_S[l] = S[l]
                    statef_B[l] = B[l]

                statef_S[faulty_state] = statef_S[faulty_state] + 1

                #Generate the keystream for the desired number of rounds
                j=0
                for keystream in range(max(val_list)+1):
                    zff = grain(S, B)
                    zf = grain(statef_S, statef_B)
                    d = zf + zff
                    if keystream == val_list[j]:
                        if d == 1:
                            result[j] += 1
                        if j < (len(val_list)-1):
                            j+=1

        array_2d.append(result)

        
    # # array printed separated with | 
    # # can be pasted in excel with delimeter
    print("Byte %s"  % (byte_index))
    for k in range(len(array_2d)):
        print ('|'.join(map(str, array_2d[k])))

    # store number of times of each outout indices = 1 in 100 tests
    csv_path = "./csv/4_byte_%s_result.csv" % (byte_index)
    with open(csv_path, mode='w') as csv_file:
        reg_writer = csv.writer(
            csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        # write the header
        # the header consists of the output indices
        reg_writer.writerow(val_list) 
        for row in array_2d: 
            reg_writer.writerow(row)
    
print("End")

Defining s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30, s31, s32, s33, s34, s35, s36, s37, s38, s39, s40, s41, s42, s43, s44, s45, s46, s47, s48, s49, s50, s51, s52, s53, s54, s55, s56, s57, s58, s59, s60, s61, s62, s63, s64, s65, s66, s67, s68, s69, s70, s71, s72, s73, s74, s75, s76, s77, s78, s79, s80, s81, s82, s83, s84, s85, s86, s87, s88, s89, s90, s91, s92, s93, s94, s95, s96, s97, s98, s99, s100, s101, s102, s103, s104, s105, s106, s107, s108, s109, s110, s111, s112, s113, s114, s115, s116, s117, s118, s119, s120, s121, s122, s123, s124, s125, s126, s127, b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28, b29, b30, b31, b32, b33, b34, b35, b36, b37, b38, b39, b40, b41, b42, b43, b44, b45, b46, b47, b48, b49, b50, b51, b52, b53, b54, b55, b56, b57, b58, b59, b60, b61, b62, b63, b64, b65, b66, b67, b68